## Video Poker using rllib and PPO

#### Group Members (alphabetical order):
 
- Fitzgerald, Ronald
- Mikhail, Susan
- Odabas, Omer


## 1) Write up the environment and test that it is working.



In [1]:
#!pip install ray[rllib]

In [2]:
#!pip install ipywidgets gputil

In [3]:
import random
import gymnasium as gym
from gymnasium import spaces, vector
import numpy as np
from ray import tune
from collections import Counter
from scipy.stats import binom, nbinom, beta, poisson, gamma, norm, geom
from ray.rllib.env import MultiAgentEnv

In [4]:
# Define global variables
deck = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace'] * 4

def initialize_deck():
    suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    ranks = ['Ace','2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King']
    deck = [(rank, suit) for suit in suits for rank in ranks]
    random.shuffle(deck)
    return deck

def partition_selected_unselected(data_list, indicator_list, max_select=None):
    if not max_select:
      max_select = len(data_list)
    unselected_list = []
    selected_list = []

    # Iterate over both lists simultaneously
    for data, indicator in zip(data_list, indicator_list):
        if indicator == 1 and len(selected_list) < max_select:
            selected_list.append(data)
        else:
            unselected_list.append(data)

    return unselected_list, selected_list

def create_card_matrix(cards):
    # Ranks and suits mapping to index
    ranks = {'Ace': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6,
             '8': 7, '9': 8, '10': 9, 'Jack': 10, 'Queen': 11, 'King': 12}
    suits = {'Clubs': 0, 'Diamonds': 1, 'Hearts': 2, 'Spades': 3}

    # Initialize the matrix with zeros
    matrix = [[0]*4 for _ in range(13)]

    # Fill the matrix based on cards input
    for rank, suit in cards:
        if rank in ranks and suit in suits:
            matrix[ranks[rank]][suits[suit]] = 1

    return matrix

def deal_hand(deck, hand_size):
    return [deck.pop() for _ in range(hand_size)]

def state_function(player_hand, deck):
    return {
        'player_hand': create_card_matrix(player_hand),
        'cards_left_in_deck': create_card_matrix(deck)
    }


# Define the rewards for each hand
rewards = {
    "Royal Flush": 800,
    "Straight Flush": 800,
    "Four of a Kind": 420,
    "Full House": 160,
    "Flush": 140,
    "Straight": 120,
    "Three of a Kind": 90,
    "Two Pair": 40,
    "Pair": 20,
    "High Card": 5
}

In [5]:
import gymnasium as gym
from collections import Counter

class CardGameEnv(gym.Env):
    def __init__(self, seed=None):
        self.hand_size = 8
        self.MAX_PLAYS = 4  # Limit the number of plays to 4
        self.MAX_DISCARDS = 3  # Limit the number of discards to 3

        self.action_space = gym.spaces.MultiDiscrete([2]*(1+self.hand_size))
        self.observation_space = gym.spaces.Dict({
            'player_hand': gym.spaces.MultiBinary([13, 4]),
            'cards_left_in_deck': gym.spaces.MultiBinary([13, 4])
        })

        self.num_discards = 0
        self.num_plays = 0
        self.deck = initialize_deck()
        self.player_hand = deal_hand(self.deck, self.hand_size)
        self.player_hand.sort()
        self.state = state_function(self.player_hand, self.deck)

    #added by Ron
    def save_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "multiturn_model.pth")
        torch.save(self.model.state_dict(), checkpoint_path)
        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "multiturn_model.pth")
        self.model.load_state_dict(torch.load(checkpoint_path))


    def reset(self, seed=None, options=None):
        self.num_discards = 0
        self.num_plays = 0
        self.deck = initialize_deck()
        self.player_hand = deal_hand(self.deck, self.hand_size)
        self.player_hand.sort()
        self.state = state_function(self.player_hand, self.deck)
        return self.state, {}

    def step(self, action):
        dec = action[0]
        if self.num_discards >= self.MAX_DISCARDS:
          dec = 1
        if self.num_plays >= self.MAX_PLAYS:
          dec = 0

        selection = action[1:]
        remaining_hand, selected_hand = partition_selected_unselected(self.player_hand, selection, max_select=5)

        if dec == 1:  # Play
            self.num_plays += 1
            hand_rank_counts = Counter([rank for rank, _ in selected_hand])
            hand_suit_counts = Counter([suit for _, suit in selected_hand])


            # Define the mapping of card ranks to their order
            card_order = {'Ace': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, 'Jack': 10, 'Queen': 11, 'King': 12}

            # Check for different hand categories
            if len(hand_rank_counts) == 5 and len(hand_suit_counts) == 1 and all(rank in ['10', 'Jack', 'Queen', 'King', 'Ace'] for rank, _ in selected_hand):
                hand_category = "Royal Flush"
            elif len(hand_rank_counts) == 5 and len(hand_suit_counts) == 1 and all(rank in card_order.keys() for rank, _ in selected_hand) and sorted(selected_hand, key=lambda x: card_order[x[0]]) in (['Ace','2', '3', '4', '5'], ['2', '3', '4', '5', '6'], ['3', '4', '5', '6', '7'], ['4', '5', '6', '7', '8'], ['5', '6', '7', '8', '9'], ['6', '7', '8', '9', '10'], ['7', '8', '9', '10', 'Jack'], ['8', '9', '10', 'Jack', 'Queen'], ['9', '10', 'Jack', 'Queen', 'King']):
                hand_category = "Straight Flush"
            elif any(count == 4 for count in hand_rank_counts.values()):
                hand_category = "Four of a Kind"
            elif any(count == 3 for count in hand_rank_counts.values()) and any(count == 2 for count in hand_rank_counts.values()):
                hand_category = "Full House"
            elif len(hand_suit_counts) == 1 and len(selected_hand) == 5:
                hand_category = "Flush"
            elif len(hand_rank_counts) == 5 and all(rank in card_order.keys() for rank, _ in selected_hand) and sorted(selected_hand, key=lambda x: card_order[x[0]]) in (['Ace','2', '3', '4', '5'], ['2', '3', '4', '5', '6'], ['3', '4', '5', '6', '7'], ['4', '5', '6', '7', '8'], ['5', '6', '7', '8', '9'], ['6', '7', '8', '9', '10'], ['7', '8', '9', '10', 'Jack'], ['8', '9', '10', 'Jack', 'Queen'], ['9', '10', 'Jack', 'Queen', 'King']):
                hand_category = "Straight"
            elif any(count == 3 for count in hand_rank_counts.values()):
                hand_category = "Three of a Kind"
            elif sum(count == 2 for count in hand_rank_counts.values()) == 2:
                hand_category = "Two Pair"
            elif any(count == 2 for count in hand_rank_counts.values()):
                hand_category = "Pair"
            else:
                hand_category = "High Card"

            reward = rewards.get(hand_category, 0)  # Reward based on hand category

        elif dec == 0:  # Discard
            self.num_discards += 1
            reward = 0  # Discarding has no reward

        self.player_hand = remaining_hand + deal_hand(self.deck, len(selected_hand))
        self.player_hand.sort()
        self.state = state_function(self.player_hand, self.deck)

        done = self.num_plays == self.MAX_PLAYS or len(self.deck) == 0

        #print(f"Player's Hand: {', '.join([f'{rank} of {suit}' for rank, suit in self.player_hand])}")
        #print(f"Action: {action}, Reward: {reward}, Done: {done}")

        return self.state, reward, done, False, {}

In [6]:
# Create an instance of the CardGameEnv class
env = CardGameEnv()
env.reset()

#Nice way to test Action Space before running RLLib
action = env.action_space.sample()
print(action)
print(env.player_hand)
state, _, _, _, _ = env.step(action)
len(state['cards_left_in_deck'])
print(env.player_hand)

[1 0 0 1 1 0 1 0 1]
[('3', 'Spades'), ('4', 'Clubs'), ('6', 'Diamonds'), ('9', 'Clubs'), ('Ace', 'Clubs'), ('Jack', 'Clubs'), ('King', 'Clubs'), ('Queen', 'Clubs')]
[('3', 'Diamonds'), ('3', 'Hearts'), ('3', 'Spades'), ('4', 'Clubs'), ('4', 'Spades'), ('Ace', 'Clubs'), ('Jack', 'Hearts'), ('King', 'Clubs')]


In [7]:
# Create an instance of the CardGameEnv class
env = CardGameEnv()
env.reset()

env.observation_space.sample()

# Testing if our observation space matches is being returned by our step function
env.observation_space.contains(state)

True

In [8]:
state, _ = env.reset()
env.observation_space.contains(state)

True

In [9]:
# Create an instance of the CardGameEnv class
env = CardGameEnv()
env.reset()

total_reward = 0  # Initialize total reward

# Game loop
while True:
    print("Player's Hand:")
    for rank, suit in env.player_hand:
        print(f"{rank} of {suit}")

    action = env.action_space.sample()

    state, reward, done, _, _ = env.step(action) # Ignoring False, {}

    total_reward += reward  # Accumulate the reward obtained in each step

    print(f"Decision made by the player: {action}")
    print(f"Total Reward after this turn: {total_reward}")

    if done:
        break

print("Game over")

Player's Hand:
3 of Clubs
3 of Spades
6 of Spades
7 of Diamonds
7 of Spades
Ace of Clubs
Jack of Diamonds
King of Hearts
Decision made by the player: [0 1 0 0 1 1 1 1 1]
Total Reward after this turn: 0
Player's Hand:
3 of Spades
5 of Diamonds
6 of Hearts
6 of Spades
9 of Diamonds
Ace of Diamonds
King of Hearts
Queen of Hearts
Decision made by the player: [0 0 0 1 0 0 1 1 1]
Total Reward after this turn: 0
Player's Hand:
10 of Diamonds
2 of Clubs
3 of Spades
5 of Diamonds
6 of Spades
8 of Clubs
8 of Diamonds
9 of Diamonds
Decision made by the player: [1 0 0 0 1 0 1 0 1]
Total Reward after this turn: 5
Player's Hand:
10 of Diamonds
2 of Clubs
3 of Spades
4 of Hearts
5 of Spades
6 of Spades
8 of Diamonds
King of Clubs
Decision made by the player: [0 0 1 1 0 0 1 0 1]
Total Reward after this turn: 5
Player's Hand:
10 of Diamonds
3 of Diamonds
4 of Hearts
5 of Spades
8 of Diamonds
8 of Hearts
King of Spades
Queen of Spades
Decision made by the player: [0 0 1 1 0 1 1 0 0]
Total Reward after t

## 2) Use RLlib to get as high an expected score as possible with 4 Plays and 3 Discards.

In [10]:
import ray
from ray import tune, air, train
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [11]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=20)

2024-04-24 13:07:24,845	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.10.11
Ray version:,2.10.0


(RolloutWorker pid=40980) 2024-04-24 13:07:44,995	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(PPO pid=25980) Trainable.setup took 15.635 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=25980) Restored on 127.0.0.1 from checkpoint: Checkpoint(filesystem=local, path=C:/Users/Fitz/ray_results/PPO_2024-04-24_01-36-22/PPO_CardGameEnv_baa10_00000_0_2024-04-24_01-36-22/checkpoint_000020)
(PPO pid=25980) 2024-04-24 13:07:49,421	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!
(PPO pid=25980) 2024-04-24 13:07:59,722	WARNING ppo.py:620 -- The mean reward returned from the environment is 31.64125 but the vf_clip_param is set to 10.0. Consider increasing it for policy: default_policy to

In [12]:
config = (PPOConfig()
          .environment(CardGameEnv)
          .framework('torch')
          .training(gamma=1)
          .rollouts(num_rollout_workers=19)
)

# Equivalent code added via .run call to use new checkpoint features, check next cell
# 
#stop = {"timesteps_total": 2000000}
#checkpoint_path = "C:\\Users\\Fitz\\ray_results\\PPO_2024-04-22_19-11-08\\PPO_CardGameEnv_bf9f4_00000_0_2024-04-22_19-11-08\\checkpoint_000009"
#
#tuner = tune.Tuner(
#    "PPO",
#    param_space=config.to_dict(),
#    run_config=air.RunConfig(
#        stop=stop,
#        checkpoint_config=train.CheckpointConfig(
#            checkpoint_frequency=50,
#            checkpoint_at_end=True)),
#)
#tuner.fit()





In [13]:
# This checkpoint path will need to be updated every run to pick up in the right place.
# Use the "best checkpoint" spit out below for reference
checkpoint_path = 'C:\\Users\\Fitz\\ray_results\\PPO_2024-04-24_01-36-22\\PPO_CardGameEnv_baa10_00000_0_2024-04-24_01-36-22\\checkpoint_000020'
stop = {"training_iteration": 3760}
analysis = tune.run(
    "PPO",  
    config=config,
    restore=checkpoint_path,
    stop=stop, 
    checkpoint_freq=50,  
    checkpoint_at_end=True
)

2024-04-24 13:07:27,394	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
C:\Users\Fitz\AppData\Roaming\Python\Python310\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
C:\Users\Fitz\AppData\Roaming\Python\Python310\site-packages\gymnasium\utils\passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
C:\Users\Fitz\AppData\Roaming\Python\Python310\site-packages\gymnasium\utils\passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


2024-04-24 13:07:47,800	WARNING logger.py:186 -- Remote file not found: C:/Users/Fitz/ray_results/PPO_2024-04-24_13-07-27/PPO_CardGameEnv_45efd_00000_0_2024-04-24_13-07-27/result.json


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
PPO_CardGameEnv_45efd_00000,15040000,"{'ObsPreprocessorConnector_ms': 0.05210326634310248, 'StateBufferConnector_ms': 0.01261289821884993, 'ViewRequirementAgentConnector_ms': 0.17591280469085135}","{'num_env_steps_sampled': 15040000, 'num_env_steps_trained': 15040000, 'num_agent_steps_sampled': 15040000, 'num_agent_steps_trained': 15040000}",{},6.64559,{},670,207.529,20,601,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.0701512441519767, 'cur_kl_coeff': 0.4500000000000001, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.901376743726832, 'policy_loss': -0.08756199602027653, 'vf_loss': 9.982358870967742, 'vf_explained_var': 3.2180099077122183e-07, 'kl': 0.014621958498513953, 'entropy': 1.6210194970971794, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 8835.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 15040000, 'num_env_steps_trained': 15040000, 'num_agent_steps_sampled': 15040000, 'num_agent_steps_trained': 15040000}",15040000,15040000,15040000,4000,353.589,15040000,4000,353.589,0,19,0,0,4000,"{'cpu_util_percent': 15.053333333333335, 'ram_util_percent': 21.399999999999995, 'gpu_util_percent0': 0.007333333333333332, 'vram_util_percent0': 0.09853515625000003}",{},{},{},"{'mean_raw_obs_processing_ms': 0.7308315490955887, 'mean_inference_ms': 4.267127123451599, 'mean_action_processing_ms': 0.19701625015073687, 'mean_env_wait_ms': 0.0971647927363242, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 11294.314, 'sample_time_ms': 1158.934, 'load_time_ms': 0.699, 'load_throughput': 5725035.318, 'learn_time_ms': 10119.278, 'learn_throughput': 395.285, 'synch_weights_time_ms': 14.61}"


2024-04-24 13:07:59,744	WARNING logger.py:186 -- Remote file not found: C:/Users/Fitz/ray_results/PPO_2024-04-24_13-07-27/PPO_CardGameEnv_45efd_00000_0_2024-04-24_13-07-27/progress.csv
2024-04-24 13:09:41,310	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-24 13:09:41,314	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to 'C:/Users/Fitz/ray_results/PPO_2024-04-24_13-07-27' in 0.0217s.
2024-04-24 13:09:41,783	INFO tune.py:1048 -- Tota

In [14]:
#Get best results
best_trial = analysis.get_best_trial("episode_reward_mean", "max", "last")
best_checkpoint = analysis.get_best_checkpoint(best_trial, "episode_reward_mean", "max")
print("Best checkpoint:", best_checkpoint)

Best checkpoint: Checkpoint(filesystem=local, path=C:/Users/Fitz/ray_results/PPO_2024-04-24_13-07-27/PPO_CardGameEnv_45efd_00000_0_2024-04-24_13-07-27/checkpoint_000000)


## 3) Use RLlib to get as high an expected score as possible with 1 Play and 1 Discard.

In [15]:
import gymnasium as gym
from collections import Counter

class CardGameEnv_One_play(gym.Env):
    def __init__(self, seed=None):
        self.hand_size = 8
        self.MAX_PLAYS = 1  # Limit the number of plays to 1
        self.MAX_DISCARDS = 1  # Limit the number of discards to 1

        self.action_space = gym.spaces.MultiDiscrete([2]*(1+self.hand_size))
        self.observation_space = gym.spaces.Dict({
            'player_hand': gym.spaces.MultiBinary([13, 4]),
            'cards_left_in_deck': gym.spaces.MultiBinary([13, 4])
        })

        self.num_discards = 0
        self.num_plays = 0
        self.deck = initialize_deck()
        self.player_hand = deal_hand(self.deck, self.hand_size)
        self.player_hand.sort()
        self.state = state_function(self.player_hand, self.deck)


    def reset(self, seed=None, options=None):
        self.num_discards = 0
        self.num_plays = 0
        self.deck = initialize_deck()
        self.player_hand = deal_hand(self.deck, self.hand_size)
        self.player_hand.sort()
        self.state = state_function(self.player_hand, self.deck)
        return self.state, {}

    #added by Ron
    def save_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "singleturn_model.pth")
        torch.save(self.model.state_dict(), checkpoint_path)
        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "single_model.pth")
        self.model.load_state_dict(torch.load(checkpoint_path))

    def step(self, action):
        dec = action[0]
        if self.num_discards >= self.MAX_DISCARDS:
          dec = 1
        if self.num_plays >= self.MAX_PLAYS:
          dec = 0

        selection = action[1:]
        remaining_hand, selected_hand = partition_selected_unselected(self.player_hand, selection, max_select=5)

        if dec == 1:  # Play
            self.num_plays += 1
            hand_rank_counts = Counter([rank for rank, _ in selected_hand])
            hand_suit_counts = Counter([suit for _, suit in selected_hand])


            # Define the mapping of card ranks to their order
            card_order = {'Ace': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, 'Jack': 10, 'Queen': 11, 'King': 12}

            # Check for different hand categories
            if len(hand_rank_counts) == 5 and len(hand_suit_counts) == 1 and all(rank in ['10', 'Jack', 'Queen', 'King', 'Ace'] for rank, _ in selected_hand):
                hand_category = "Royal Flush"
            elif len(hand_rank_counts) == 5 and len(hand_suit_counts) == 1 and all(rank in card_order.keys() for rank, _ in selected_hand) and sorted(selected_hand, key=lambda x: card_order[x[0]]) in (['Ace','2', '3', '4', '5'], ['2', '3', '4', '5', '6'], ['3', '4', '5', '6', '7'], ['4', '5', '6', '7', '8'], ['5', '6', '7', '8', '9'], ['6', '7', '8', '9', '10'], ['7', '8', '9', '10', 'Jack'], ['8', '9', '10', 'Jack', 'Queen'], ['9', '10', 'Jack', 'Queen', 'King']):
                hand_category = "Straight Flush"
            elif any(count == 4 for count in hand_rank_counts.values()):
                hand_category = "Four of a Kind"
            elif any(count == 3 for count in hand_rank_counts.values()) and any(count == 2 for count in hand_rank_counts.values()):
                hand_category = "Full House"
            elif len(hand_suit_counts) == 1 and len(selected_hand) == 5:
                hand_category = "Flush"
            elif len(hand_rank_counts) == 5 and all(rank in card_order.keys() for rank, _ in selected_hand) and sorted(selected_hand, key=lambda x: card_order[x[0]]) in (['Ace','2', '3', '4', '5'], ['2', '3', '4', '5', '6'], ['3', '4', '5', '6', '7'], ['4', '5', '6', '7', '8'], ['5', '6', '7', '8', '9'], ['6', '7', '8', '9', '10'], ['7', '8', '9', '10', 'Jack'], ['8', '9', '10', 'Jack', 'Queen'], ['9', '10', 'Jack', 'Queen', 'King']):
                hand_category = "Straight"
            elif any(count == 3 for count in hand_rank_counts.values()):
                hand_category = "Three of a Kind"
            elif sum(count == 2 for count in hand_rank_counts.values()) == 2:
                hand_category = "Two Pair"
            elif any(count == 2 for count in hand_rank_counts.values()):
                hand_category = "Pair"
            else:
                hand_category = "High Card"

            reward = rewards.get(hand_category, 0)  # Reward based on hand category

        elif dec == 0:  # Discard
            self.num_discards += 1
            reward = 0  # Discarding has no reward

        self.player_hand = remaining_hand + deal_hand(self.deck, len(selected_hand))
        self.player_hand.sort()
        self.state = state_function(self.player_hand, self.deck)

        done = self.num_plays == self.MAX_PLAYS or len(self.deck) == 0

        #print(f"Player's Hand: {', '.join([f'{rank} of {suit}' for rank, suit in self.player_hand])}")
        #print(f"Action: {action}, Reward: {reward}, Done: {done}")

        return self.state, reward, done, False, {}

In [16]:
# Create an instance of the CardGameEnv class
env = CardGameEnv_One_play()
env.reset()

#Nice way to test Action Space before running RLLib
action = env.action_space.sample()
print(action)
print(env.player_hand)
state, _, _, _, _ = env.step(action)
len(state['cards_left_in_deck'])
print(env.player_hand)

[1 1 1 0 0 0 1 1 1]
[('10', 'Clubs'), ('3', 'Clubs'), ('3', 'Spades'), ('4', 'Diamonds'), ('Ace', 'Diamonds'), ('Ace', 'Spades'), ('Queen', 'Clubs'), ('Queen', 'Diamonds')]
[('3', 'Hearts'), ('3', 'Spades'), ('4', 'Diamonds'), ('5', 'Clubs'), ('6', 'Clubs'), ('Ace', 'Diamonds'), ('King', 'Spades'), ('Queen', 'Hearts')]


In [17]:
# Create an instance of the CardGameEnv_One_play class
env = CardGameEnv_One_play()
env.reset()

env.observation_space.sample()

# Testing if our observation space matches is being returned by our step function
env.observation_space.contains(state)

True

In [18]:
state, _ = env.reset()
env.observation_space.contains(state)

True

In [19]:
# Create an instance of the CardGameEnv_One_play class
env = CardGameEnv_One_play()
env.reset()

total_reward = 0  # Initialize total reward

# Game loop
while True:
    print("Player's Hand:")
    for rank, suit in env.player_hand:
        print(f"{rank} of {suit}")

    action = env.action_space.sample()

    state, reward, done, _, _ = env.step(action) # Ignoring False, {}

    total_reward += reward  # Accumulate the reward obtained in each step

    print(f"Decision made by the player: {action}")
    print(f"Total Reward after this turn: {total_reward}")

    if done:
        break

print("Game over")

Player's Hand:
2 of Diamonds
2 of Spades
3 of Clubs
5 of Spades
8 of Clubs
Ace of Clubs
Ace of Hearts
Queen of Diamonds
Decision made by the player: [1 0 0 0 0 1 1 1 0]
Total Reward after this turn: 20
Game over


In [20]:
one_play_config = (PPOConfig()
          .environment(CardGameEnv_One_play)
          .framework('torch')
          .training(gamma=1)
          .rollouts(num_rollout_workers=19)
)

#stop = {"timesteps_total": 1000000}
#
#tuner = tune.Tuner(
#    "PPO",
#    param_space=config.to_dict(),
#    run_config=air.RunConfig(stop=stop, checkpoint_config=train.CheckpointConfig(checkpoint_frequency=50, checkpoint_at_end=True)),
#)
#
#tuner.fit()

one_play_checkpoint_path = 'C:\\Users\\Fitz\\ray_results\\PPO_2024-04-23_14-04-12\\PPO_CardGameEnv_One_play_092a9_00000_0_2024-04-23_14-04-12\\checkpoint_000012'
one_play_stop = {"training_iteration": 910}
one_play_analysis = tune.run(
    "PPO",  # or use ppo.PPOTrainer if you prefer not to use the string identifier
    config=one_play_config,
    restore=one_play_checkpoint_path,
    stop=one_play_stop, 
    checkpoint_freq=50,  
    checkpoint_at_end=True
)


2024-04-24 13:09:41,937	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-04-24 13:10:03,167	WARNING logger.py:186 -- Remote file not found: C:/Users/Fitz/ray_results/PPO_2024-04-24_13-09-41/PPO_CardGameEnv_One_play_9619c_00000_0_2024-04-24_13-09-42/result.json


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
PPO_CardGameEnv_One_play_9619c_00000,3640000,"{'ObsPreprocessorConnector_ms': 0.050044466882186944, 'StateBufferConnector_ms': 0.021985718132505213, 'ViewRequirementAgentConnector_ms': 0.18974060517755562}","{'num_env_steps_sampled': 3640000, 'num_env_steps_trained': 3640000, 'num_agent_steps_sampled': 3640000, 'num_agent_steps_trained': 3640000}",{},1.66431,{},420,43.0029,5,2401,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.7581511560947665, 'cur_kl_coeff': 0.4500000000000001, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.902824258291593, 'policy_loss': -0.07542646511267591, 'vf_loss': 9.96975806451613, 'vf_explained_var': -1.2568248215542044e-07, 'kl': 0.01887255935804551, 'entropy': 1.8798622541530159, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 8835.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 3640000, 'num_env_steps_trained': 3640000, 'num_agent_steps_sampled': 3640000, 'num_agent_steps_trained': 3640000}",3640000,3640000,3640000,4000,347.743,3640000,4000,347.743,0,19,0,0,4000,"{'cpu_util_percent': 18.057142857142857, 'ram_util_percent': 21.564285714285717, 'gpu_util_percent0': 0.009285714285714286, 'vram_util_percent0': 0.10026506696428573}",{},{},{},"{'mean_raw_obs_processing_ms': 1.356127354236026, 'mean_inference_ms': 4.409880477183959, 'mean_action_processing_ms': 0.2107561848994905, 'mean_env_wait_ms': 0.09755011844898423, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 11630.73, 'sample_time_ms': 1333.874, 'load_time_ms': 0.399, 'load_throughput': 10020436.003, 'learn_time_ms': 10280.071, 'learn_throughput': 389.102, 'synch_weights_time_ms': 15.699}"


2024-04-24 13:10:15,342	WARNING logger.py:186 -- Remote file not found: C:/Users/Fitz/ray_results/PPO_2024-04-24_13-09-41/PPO_CardGameEnv_One_play_9619c_00000_0_2024-04-24_13-09-42/progress.csv
2024-04-24 13:12:00,267	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to 'C:/Users/Fitz/ray_results/PPO_2024-04-24_13-09-41' in 0.0255s.
2024-04-24 13:12:00,333	INFO tune.py:1048 -- Total run time: 138.40 seconds (138.24 seconds for the tuning loop).


Best checkpoint: Checkpoint(filesystem=local, path=C:/Users/Fitz/ray_results/PPO_2024-04-24_13-07-27/PPO_CardGameEnv_45efd_00000_0_2024-04-24_13-07-27/checkpoint_000000)


In [21]:
one_play_best_trial = one_play_analysis.get_best_trial("episode_reward_mean", "max", "last")
one_play_best_checkpoint = one_play_analysis.get_best_checkpoint(one_play_best_trial, "episode_reward_mean", "max")
print("Best checkpoint:", one_play_best_checkpoint)

Best checkpoint: Checkpoint(filesystem=local, path=C:/Users/Fitz/ray_results/PPO_2024-04-24_13-09-41/PPO_CardGameEnv_One_play_9619c_00000_0_2024-04-24_13-09-42/checkpoint_000000)


## Thank you!